# Aug 2023 BTC crash analysis - 15 minutes data

***

# Import libraries and set some intial paramenters

## Import relevant libraries

In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
from dotenv import load_dotenv

import json
from datetime import datetime, timedelta, date
import plotly.graph_objects as go
import plotly.express as px
import os

import sys
sys.path.append('../src')

import cryptonews as cn


## Set some initial parameters

In [ ]:
load_dotenv(dotenv_path='../config_files/.env')

# Get Data

## [OPTIONAL] Download the data from the API's
Only if you have the API keys for `cryptonews-api` and `binance-api` and you want to download the data from the API's.

### Download Binance BTCUSDT data

In [ ]:
asset_ticket = "BTCUSDT"
timestamp = 'Client.KLINE_INTERVAL_15MINUTE'
# start_date = "01 Aug, 2023"
# end_date = "20 Aug, 2023"
start_date = "16 Aug, 2023"
end_date = "18 Aug, 2023"

In [ ]:
# Create the Binance API client
client = Client(os.environ["BINANCE_API_KEY"], os.environ["BINANCE_SECRET_KEY"])

In [ ]:
klines = client.get_historical_klines(asset_ticket, eval(timestamp), start_date, end_date)

In [ ]:
df_klines = pd.DataFrame(klines, columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Convert 'object' to float pandas
df_klines['open'] = pd.to_numeric(df_klines['open'])
df_klines['high'] = pd.to_numeric(df_klines['high'])
df_klines['low'] = pd.to_numeric(df_klines['low'])
df_klines['close'] = pd.to_numeric(df_klines['close'])
df_klines['volume'] = pd.to_numeric(df_klines['volume'])

# Convert the 'open_time' and 'close_time' to a Pandas DataTime format
df_klines['formatted_open_time'] = pd.to_datetime(df_klines['open_time'], infer_datetime_format=True, unit="ms")

# Converting Date Column to DateTime Type
df_klines['date'] = pd.to_datetime(df_klines['formatted_open_time'])
df_klines['date'] = pd.to_datetime(df_klines['date']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_klines['date'] = pd.to_datetime(df_klines['date'])

# Get the columns we need
df_klines = df_klines[['date', 'open', 'high', 'low', 'close', 'volume']]

# Export the data to a CSV file
df_klines.to_csv(f"../data/{asset_ticket}_15min_data_from_{datetime.strptime(start_date,'%d %b, %Y').strftime('%Y%m%d')}_to_{datetime.strptime(end_date,'%d %b, %Y').strftime('%Y%m%d')}.csv", index=False)

### Download Crypto News data

In [ ]:
ticket = "BTC"
sd = date(2023, 8, 1)
ed = date(2023, 8, 20)

In [ ]:
cn.get_cryptonews('BTC', sd, ed)

## Import data from local files

In [2]:
btc = pd.read_csv('../data/BTCUSDT_15min_data_from_20230816_to_20230818.csv')
btc['date'] = pd.to_datetime(btc['date'])

In [3]:
btc.head()

,date,open,high,low,close,volume
0,2023-08-16 00:00:00,29200.01,29227.56,29196.21,29227.55,105.95848
1,2023-08-16 00:15:00,29227.55,29237.27,29216.18,29216.18,91.71733
2,2023-08-16 00:30:00,29216.19,29216.19,29200.48,29203.00,174.88783
3,2023-08-16 00:45:00,29203.00,29259.85,29199.99,29224.37,266.36770
4,2023-08-16 01:00:00,29224.37,29229.75,29209.96,29226.13,210.20481


In [18]:
news = pd.read_csv('../data/formatted_cryptonews_BTC_from_20230801_to_20230820.csv')
news['utc_datetime'] = pd.to_datetime(news['utc_datetime'])
news = news[(news['utc_datetime'] >= '2023-08-16 00:00:00') & (news['utc_datetime'] <= '2023-08-18 00:00:00')]
news = news.sort_values(by='utc_datetime', ascending=True)

In [19]:
news.head()

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers,news_id,rank_score,eventid,datetime,utc_zone,utc_datetime
1247,https://www.newsbtc.com/bitcoin-news/this-drea...,https://crypto.snapi.dev/images/v1/i/h/bitcoin...,This Dreaded Candlestick Formation Just Printe...,"Bitcoin, the most valuable crypto asset, is at...",NewsBTC,"Tue, 15 Aug 2023 20:00:53 -0400",['pricemovement'],Negative,Article,['BTC'],348587,4.96,NaN,2023-08-15 20:00:53,-400,2023-08-16 00:00:53
1270,https://investorplace.com/2023/08/7-cryptos-on...,https://crypto.snapi.dev/images/v1/5/y/5yh2-34...,7 Cryptos on Watch as Blockchain Miners Lose T...,"Once again, cryptos have entered into a frustr...",Investorplace,"Tue, 15 Aug 2023 21:28:15 -0400",['mining'],Negative,Article,"['BTC', 'ETH', 'USDT', 'ADA', 'DOGE', 'SOL', '...",348603,4.37,NaN,2023-08-15 21:28:15,-400,2023-08-16 01:28:15
1203,https://ambcrypto.com/assessing-bitcoin-ordina...,https://crypto.snapi.dev/images/v1/z/i/bitcoin...,Assessing Bitcoin Ordinals' crest and trough s...,Here's how Bitcoin Ordinals fared over the las...,AMBCrypto,"Tue, 15 Aug 2023 21:30:53 -0400",[],Neutral,Article,['BTC'],348602,6.18,NaN,2023-08-15 21:30:53,-400,2023-08-16 01:30:53
1244,https://thecurrencyanalytics.com/bitcoin/polit...,https://crypto.snapi.dev/images/v1/l/j/bitcoin...,Political Winds and Bitcoin ETFs: A Former SEC...,"In a twist that has caught many by surprise, a...",The Currency Analytics,"Tue, 15 Aug 2023 21:45:55 -0400","['regulations', 'priceforecast']",Positive,Article,['BTC'],348606,5.10,NaN,2023-08-15 21:45:55,-400,2023-08-16 01:45:55
1236,https://www.crowdfundinsider.com/2023/08/21143...,https://crypto.snapi.dev/images/v1/p/a/cryptoc...,Bitcoin (BTC) Mining Firm Riot Platforms Relea...,"Riot Platforms, Inc. (NASDAQ: RIOT), which cla...",CrowdFundInsider,"Tue, 15 Aug 2023 21:56:11 -0400",['mining'],Positive,Article,['BTC'],348610,5.46,NaN,2023-08-15 21:56:11,-400,2023-08-16 01:56:11


***
# Analytics

* _Context_: during the third week of August 2023, BTC's price plummeted.

* _Hypothesis_: could we have foreseen such drastic downward trend with news sentiment data?

In [20]:
fig = go.Figure(data=[go.Candlestick(x=btc['date'],
                open=btc['open'], high=btc['high'],
                low=btc['low'], close=btc['close'])
                     ])

fig.update_layout(
     title="<b>Bitcoin USDT Price - August 2023</b>",
     yaxis_title="Price in USDT",
     xaxis_title="Date",
     xaxis_rangeslider_visible=False,
     template="ggplot2"
     )
fig.update_xaxes(type='category')

fig.show()

In [21]:
# Function to round time to the next 15 minutes
def round_time(dt):
    minute = (dt.minute // 15 + 1) * 15 % 60
    second = 0
    hour = dt.hour
    if minute == 0:
        hour = (hour + 1) % 24
    return pd.Timestamp(year=dt.year, month=dt.month, day=dt.day, hour=hour, minute=minute, second=second)

In [22]:
news_ = news.copy()
news_['utc_datetime'] = pd.to_datetime(news_['utc_datetime'])
news_['round_15min_utc_time'] = news_['utc_datetime'].apply(round_time)

In [23]:
news_.columns

Index(['news_url', 'image_url', 'title', 'text', 'source_name', 'date',
       'topics', 'sentiment', 'type', 'tickers', 'news_id', 'rank_score',
       'eventid', 'datetime', 'utc_zone', 'utc_datetime',
       'round_15min_utc_time'],
      dtype='object')

In [24]:
news_ = news_[['utc_datetime', 'round_15min_utc_time', 'title', 'text', 'sentiment', 'rank_score']]

In [25]:
news_.head()

,utc_datetime,round_15min_utc_time,title,text,sentiment,rank_score
1247,2023-08-16 00:00:53,2023-08-16 00:15:00,This Dreaded Candlestick Formation Just Printe...,"Bitcoin, the most valuable crypto asset, is at...",Negative,4.96
1270,2023-08-16 01:28:15,2023-08-16 01:30:00,7 Cryptos on Watch as Blockchain Miners Lose T...,"Once again, cryptos have entered into a frustr...",Negative,4.37
1203,2023-08-16 01:30:53,2023-08-16 01:45:00,Assessing Bitcoin Ordinals' crest and trough s...,Here's how Bitcoin Ordinals fared over the las...,Neutral,6.18
1244,2023-08-16 01:45:55,2023-08-16 02:00:00,Political Winds and Bitcoin ETFs: A Former SEC...,"In a twist that has caught many by surprise, a...",Positive,5.10
1236,2023-08-16 01:56:11,2023-08-16 02:00:00,Bitcoin (BTC) Mining Firm Riot Platforms Relea...,"Riot Platforms, Inc. (NASDAQ: RIOT), which cla...",Positive,5.46


In [26]:
all_utc_datetime = pd.date_range(start=btc['date'].min(), end=btc['date'].max(), freq='15T')

In [27]:
all_utc_datetime

DatetimeIndex(['2023-08-16 00:00:00', '2023-08-16 00:15:00',
               '2023-08-16 00:30:00', '2023-08-16 00:45:00',
               '2023-08-16 01:00:00', '2023-08-16 01:15:00',
               '2023-08-16 01:30:00', '2023-08-16 01:45:00',
               '2023-08-16 02:00:00', '2023-08-16 02:15:00',
               ...
               '2023-08-17 21:45:00', '2023-08-17 22:00:00',
               '2023-08-17 22:15:00', '2023-08-17 22:30:00',
               '2023-08-17 22:45:00', '2023-08-17 23:00:00',
               '2023-08-17 23:15:00', '2023-08-17 23:30:00',
               '2023-08-17 23:45:00', '2023-08-18 00:00:00'],
              dtype='datetime64[ns]', length=193, freq='15T')

In [28]:
all_date_news = pd.DataFrame()
all_date_news['all_utc_datetime'] = all_utc_datetime
all_date_news = all_date_news.merge(news_, how='left', left_on='all_utc_datetime', right_on='round_15min_utc_time')

# full fille the NaN rows with previous values
all_date_news = all_date_news.fillna(method='ffill')
all_date_news.head(20)

,all_utc_datetime,utc_datetime,round_15min_utc_time,title,text,sentiment,rank_score
0,2023-08-16 00:00:00,2023-08-16 23:52:10,2023-08-16 00:00:00,Prominent Analyst Foresees Bitcoin Surging to ...,"On a recent CNBC ‘Squawk Box' segment, Thomas ...",Positive,4.73
1,2023-08-16 00:15:00,2023-08-16 00:00:53,2023-08-16 00:15:00,This Dreaded Candlestick Formation Just Printe...,"Bitcoin, the most valuable crypto asset, is at...",Negative,4.96
2,2023-08-16 00:30:00,2023-08-16 00:00:53,2023-08-16 00:15:00,This Dreaded Candlestick Formation Just Printe...,"Bitcoin, the most valuable crypto asset, is at...",Negative,4.96
3,2023-08-16 00:45:00,2023-08-16 00:00:53,2023-08-16 00:15:00,This Dreaded Candlestick Formation Just Printe...,"Bitcoin, the most valuable crypto asset, is at...",Negative,4.96
4,2023-08-16 01:00:00,2023-08-16 00:00:53,2023-08-16 00:15:00,This Dreaded Candlestick Formation Just Printe...,"Bitcoin, the most valuable crypto asset, is at...",Negative,4.96
5,2023-08-16 01:15:00,2023-08-16 00:00:53,2023-08-16 00:15:00,This Dreaded Candlestick Formation Just Printe...,"Bitcoin, the most valuable crypto asset, is at...",Negative,4.96
6,2023-08-16 01:30:00,2023-08-16 01:28:15,2023-08-16 01:30:00,7 Cryptos on Watch as Blockchain Miners Lose T...,"Once again, cryptos have entered into a frustr...",Negative,4.37
7,2023-08-16 01:45:00,2023-08-16 01:30:53,2023-08-16 01:45:00,Assessing Bitcoin Ordinals' crest and trough s...,Here's how Bitcoin Ordinals fared over the las...,Neutral,6.18
8,2023-08-16 02:00:00,2023-08-16 01:45:55,2023-08-16 02:00:00,Political Winds and Bitcoin ETFs: A Former SEC...,"In a twist that has caught many by surprise, a...",Positive,5.10
9,2023-08-16 02:00:00,2023-08-16 01:56:11,2023-08-16 02:00:00,Bitcoin (BTC) Mining Firm Riot Platforms Relea...,"Riot Platforms, Inc. (NASDAQ: RIOT), which cla...",Positive,5.46


In [29]:
sents = all_date_news.groupby('all_utc_datetime')['sentiment'].value_counts(normalize=True).to_frame('count')
sents['count'] = round(sents['count']*100)
sents = sents.reset_index()

fig = px.bar(
    sents, 
    x='all_utc_datetime', 
    y='count', 
    text=[f'{c}%' for c in sents['count']],
    color='sentiment', 
    color_discrete_map={'Positive': '#77dd77',
                        'Negative': '#ff6961',
                        'Neutral': '#ffb347'
                        }
    )

fig.update_layout(
     title="<b>Cryptonews Sentiment - August 2023</b>",
     yaxis_title="Count per Sentiment",
     xaxis_title="Date",
     template="ggplot2"
     )

# fig.update_xaxes(type='category')
fig.add_vline(x='2023-08-17 21:30:00',line_width=4, line_dash="dash", line_color="black")
fig.show()

In [30]:
sents = news_.groupby('round_15min_utc_time')['sentiment'].value_counts(normalize=True).to_frame('count')
sents['count'] = round(sents['count']*100)
sents = sents.reset_index()

fig = px.bar(
    sents, 
    x='round_15min_utc_time', 
    y='count', 
    text=[f'{c}%' for c in sents['count']],
    color='sentiment', 
    color_discrete_map={'Positive': '#77dd77',
                        'Negative': '#ff6961',
                        'Neutral': '#ffb347'
                        }
    )

fig.update_layout(
     title="<b>Cryptonews Sentiment - August 2023</b>",
     yaxis_title="Count per Sentiment",
     xaxis_title="Date",
     template="ggplot2"
     )

# fig.update_xaxes(type='category')
fig.add_vline(x='2023-08-17 21:30:00',line_width=4, line_dash="dash", line_color="black")
fig.show()

Which does not seem to predict the downward trend. 

* Which news are related to whales offloading?

* What if we weight the sentiments based on rank?

In [31]:
keywords = ['Elon', 'Musk', 'SpaceX', 'Whale', 'whale']
kwstr = '|'.join(keywords)
mask = all_date_news[['title', 'text']].stack().str.contains(kwstr).unstack().any(axis=1)


In [32]:
whales = all_date_news[mask].sort_values('all_utc_datetime')
whales_sent = whales.groupby('all_utc_datetime')['sentiment'].value_counts().to_frame('count').reset_index()

In [33]:
fig = px.bar(
    whales_sent, 
    x='all_utc_datetime', 
    y='count', 
    # text=[f'{c}%' for c in sents['count']],
    color='sentiment', 
    color_discrete_map={'Positive': '#77dd77',
                        'Negative': '#ff6961',
                        'Neutral': '#ffb347'
                        }
    )

fig.update_layout(
     title="<b>Cryptonews Whales-Related Sentiment - August 2023</b>",
     yaxis_title="Count per Sentiment",
     xaxis_title="Date",
     template="ggplot2"
     )

# fig.update_xaxes(type='category')
fig.add_vline(x='2023-08-17 21:30:00',line_width=2, line_dash="dash", line_color="black")
fig.show()

Still inconclusive.

In [48]:
news_2 = all_date_news.copy()

condlist = [
    news_2['sentiment']=="Positive",
    news_2['sentiment']=="Neutral",
    news_2['sentiment']=="Negative"
]

choices = [1, 0, -1]

news_2['polarity'] = np.select(condlist,choices)

news_2.rank_score = pd.to_numeric(news_2.rank_score)

wm = lambda x: np.average(x, weights=news_2.loc[x.index, 'rank_score'])

weighted = news_2.groupby(['all_utc_datetime']).agg(weighted_avg=('polarity', wm))

In [72]:
# Create a column with the cumulative sum of the weighted average starting by 100
weighted['weighted_avg_index'] = weighted['weighted_avg'].cumsum() + 100

In [73]:
fig = px.line(
    weighted.reset_index()[1:],
    x='all_utc_datetime',
    y='weighted_avg',
    markers=True
)

fig.update_layout(
     title="<b>Cryptonews Weighted Sentiment - August 2023</b>",
     yaxis_title="Weighted Sentiment",
     xaxis_title="Date",
     template="ggplot2"
     )

# fig.update_xaxes(type='category')
fig.add_vline(x='2023-08-17 21:30:00',line_width=2, line_dash="dash", line_color="black")
fig.show()

In [74]:
fig = px.line(
    weighted.reset_index()[1:],
    x='all_utc_datetime',
    y='weighted_avg_index',
    markers=True
)

fig.update_layout(
     title="<b>Cryptonews Weighted Index Sentiment - August 2023</b>",
     yaxis_title="Weighted Sentiment",
     xaxis_title="Date",
     template="ggplot2"
     )

# fig.update_xaxes(type='category')
fig.add_vline(x='2023-08-17 21:30:00',line_width=2, line_dash="dash", line_color="black")
fig.show()

In [75]:
# What about pct change?
weighted['pct_chg'] = weighted['weighted_avg_index'].pct_change()
weighted['pct_chg'] = weighted['pct_chg'].fillna(0)

In [80]:
fig = px.line(
    weighted.reset_index()[2:],
    x='all_utc_datetime',
    y='pct_chg',
    # text=[f'{int(round(p*100,0))}%' for p in weighted['pct_chg'][2:]],
    markers=True
)

fig.update_layout(
     title="<b>Cryptonews Weighted Sentiment Percentual Change - August 2023</b>",
     yaxis_title="Weighted Sentiment",
     xaxis_title="Date",
     template="ggplot2",
     width=1500,
     height=500
     )

# fig.update_xaxes(type='category')

# fig.update_traces(marker={'size':18, 'symbol':'square'})
# fig.update_traces(textposition='middle center', textfont_size=6, textfont_color='white')
fig.add_vline(x='2023-08-17 21:30:00',line_width=2, line_dash="dash", line_color="black")
fig.add_hline(y=0,line_width=2, line_color="black")
fig.show()

Weighted sentiment seems to work better somewhat. Further conclusions would require going beyond analysis.